In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import numpy as np
import seaborn as sns
import pandas as pd
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist
from scipy import linalg
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 10)

## Работа с данными ДФЛ, строим таблицу со спредами и объединяем банки у которых низкая доля рынка (менее процента) в один банк с названием lb для рашения проблемы размерности VAR


In [2]:
data = pd.read_csv('ru_banks_metrics.csv')

In [3]:
data.head()

,report_date,MosPrime_1Month,PnL_rur_deCum,PnL_usd_deCum,volume_rur,volume_usd,implied_ann_rate_rur,implied_ann_rate_usd,spread_rur,bank_group_id,sector_id
0,2007-07-01 00:00:00,0.041193,0.011135,0.0,0.493424,0.0,0.090267,NaN,0.049074,6,52
1,2007-10-01 00:00:00,0.070900,0.013234,0.0,0.578747,0.0,0.091467,NaN,0.020567,6,52
2,2008-01-01 00:00:00,0.064773,0.007423,0.0,0.371901,0.0,0.079838,NaN,0.015065,6,52
3,2008-04-01 00:00:00,0.063833,0.003259,0.0,0.142167,0.0,0.091695,NaN,0.027862,6,52
4,2008-07-01 00:00:00,0.051083,0.002471,0.0,0.104901,0.0,0.094222,NaN,0.043139,6,52


In [4]:
len(data.sector_id.value_counts())

18

In [5]:
data_grouped = data.volume_rur[data.sector_id]

In [6]:
data.bank_group_id.value_counts()

0     817
2     805
1     800
3     799
26    794
     ... 
55    225
73    219
67    172
69    117
60     66
Name: bank_group_id, Length: 85, dtype: int64

In [7]:
sector_id_1

NameError: name 'sector_id_1' is not defined

In [7]:
data_dfl = data[data.sector_id == 14]

In [8]:
data_dfl_spreads = data_dfl[['report_date','spread_rur','bank_group_id','volume_rur']]
data_dfl_spreads_grouped = data_dfl_spreads.groupby('bank_group_id')

In [9]:
data_dfl_spreads = data_dfl_spreads.reset_index()

In [10]:
data_dfl_spreads

,index,report_date,spread_rur,bank_group_id,volume_rur
0,623,2008-04-01 00:00:00,0.016129,6,8.552250
1,624,2008-07-01 00:00:00,0.027548,6,9.057004
2,625,2008-10-01 00:00:00,-0.005525,6,9.022148
3,626,2009-01-01 00:00:00,-0.143744,6,6.408012
4,627,2009-04-01 00:00:00,-0.052283,6,3.750384
...,...,...,...,...,...
3286,46269,2019-01-01 00:00:00,-0.008223,57,8.316972
3287,46270,2019-04-01 00:00:00,-0.015592,57,6.408528
3288,46271,2019-07-01 00:00:00,-0.006989,57,4.946177
3289,46272,2019-10-01 00:00:00,0.004707,57,5.257895


In [11]:
spread_rur = pd.DataFrame()
for i in data_dfl_spreads.bank_group_id.unique():
    spread_rur[f'spread_{i}'] = data_dfl_spreads.spread_rur[data_dfl_spreads.bank_group_id == i].reset_index(drop=True)

In [12]:
spread_rur_nanfree = spread_rur.dropna(axis=1)

In [13]:
spread_rur_nanfree

,spread_6,spread_0,spread_50,spread_4,spread_18,spread_34,spread_10,spread_3,spread_21,spread_7,...,spread_15,spread_72,spread_40,spread_42,spread_75,spread_62,spread_44,spread_41,spread_26,spread_71
0,0.016129,0.027603,0.031042,0.010090,0.033186,0.040709,0.043200,0.020573,0.050018,0.019281,...,0.037459,0.020984,0.019597,0.019508,0.035194,0.028431,0.005117,0.017071,0.040588,0.043219
1,0.027548,0.051623,0.011235,0.022688,0.045468,0.054527,0.059770,0.032715,0.063333,0.027570,...,0.049287,0.070998,0.032294,0.027579,0.057213,0.041468,0.012267,0.029513,0.052065,0.049998
2,-0.005525,0.012787,0.064318,-0.005088,0.017041,0.014027,0.035407,0.002691,0.034826,-0.004452,...,0.015916,0.028178,0.007613,0.017492,0.039027,0.029777,-0.028500,-0.003593,0.027110,0.023036
3,-0.143744,-0.130917,-0.180260,-0.140891,-0.113809,0.026829,-0.125518,-0.129540,-0.079387,-0.158759,...,-0.138198,-0.096707,-0.128782,-0.145707,-0.116532,-0.137658,-0.145772,-0.143798,-0.111039,-0.105107
4,-0.052283,-0.059153,-0.093000,-0.049791,-0.061243,0.002649,-0.050089,-0.059392,-0.018271,-0.069093,...,-0.058999,-0.047482,-0.055213,-0.068296,-0.075065,-0.045542,-0.076403,-0.076280,-0.054738,-0.039117
5,0.041213,0.007600,-0.008760,-0.003837,0.003791,0.058972,0.016438,0.012884,0.050460,0.033410,...,0.014860,0.019316,0.025173,0.025626,0.004890,0.031041,0.009309,-0.001682,0.008840,0.033058
6,0.028134,0.057094,0.007306,0.014946,0.017860,0.069342,0.030761,0.030251,0.068089,0.033327,...,0.036069,0.033387,0.042880,0.027927,0.022483,0.040844,0.007888,0.006457,0.023849,0.051403
7,0.052090,0.059752,0.031002,0.039784,0.047287,0.107573,0.062406,0.056910,0.096037,0.052786,...,0.070548,0.060979,0.072216,0.051622,0.039046,0.066630,0.042477,0.035597,0.053434,0.081845
8,0.061919,0.076043,0.063181,0.053095,0.155359,0.115978,0.087497,0.074664,0.108014,0.581427,...,0.092942,0.082335,0.085159,0.069740,0.082697,0.081567,0.058519,0.054627,0.075227,0.097716
9,0.049665,0.075459,0.068270,0.053936,0.077203,0.114903,0.091224,0.076913,0.102753,0.084376,...,0.094426,0.084611,0.080304,0.068913,0.081304,0.077228,0.056265,0.053133,0.077232,0.098252


## Correlation analysis

In [14]:
correlation_matrix = spread_rur_nanfree.corr(method='spearman')
correlation_matrix

,spread_6,spread_0,spread_50,spread_4,spread_18,spread_34,spread_10,spread_3,spread_21,spread_7,...,spread_15,spread_72,spread_40,spread_42,spread_75,spread_62,spread_44,spread_41,spread_26,spread_71
spread_6,1.000000,0.844659,0.771277,0.825228,0.829896,0.830873,0.858011,0.880591,0.904690,0.888841,...,0.807968,0.843899,0.890795,0.897851,0.792119,0.874620,0.748263,0.736539,0.872232,0.889492
spread_0,0.844659,1.000000,0.817195,0.920756,0.807425,0.858554,0.908489,0.914025,0.907295,0.840100,...,0.911528,0.914785,0.933239,0.894160,0.827508,0.849327,0.630265,0.733825,0.918150,0.917825
spread_50,0.771277,0.817195,1.000000,0.824685,0.778550,0.749240,0.884607,0.867673,0.826096,0.828593,...,0.769431,0.820560,0.778441,0.822840,0.846722,0.718519,0.526379,0.709835,0.884281,0.778658
spread_4,0.825228,0.920756,0.824685,1.000000,0.801129,0.843248,0.914459,0.962223,0.866153,0.868541,...,0.855189,0.940404,0.912397,0.839991,0.833261,0.806665,0.614525,0.784737,0.954190,0.866044
spread_18,0.829896,0.807425,0.778550,0.801129,1.000000,0.819909,0.909900,0.836518,0.901433,0.830764,...,0.861919,0.872232,0.869192,0.922167,0.947351,0.885258,0.828919,0.736539,0.875488,0.913265
spread_34,0.830873,0.858554,0.749240,0.843248,0.819909,1.000000,0.888732,0.873860,0.912397,0.853018,...,0.829353,0.902301,0.916739,0.887755,0.809271,0.859748,0.694637,0.722644,0.884716,0.906209
spread_10,0.858011,0.908489,0.884607,0.914459,0.909900,0.888732,1.000000,0.942032,0.952888,0.930960,...,0.911420,0.953647,0.917607,0.936604,0.933673,0.900564,0.701151,0.800912,0.971016,0.931394
spread_3,0.880591,0.914025,0.867673,0.962223,0.836518,0.873860,0.942032,1.000000,0.897416,0.936496,...,0.859205,0.940621,0.908923,0.893508,0.854212,0.838363,0.643726,0.794507,0.967325,0.889818
spread_21,0.904690,0.907295,0.826096,0.866153,0.901433,0.912397,0.952888,0.897416,1.000000,0.905232,...,0.869084,0.924446,0.944746,0.942358,0.864742,0.935302,0.728615,0.712006,0.938667,0.945397
spread_7,0.888841,0.840100,0.828593,0.868541,0.830764,0.853018,0.930960,0.936496,0.905232,1.000000,...,0.840208,0.891663,0.862571,0.880265,0.829679,0.853343,0.670321,0.783000,0.914894,0.884064


## Кластеризация


In [15]:
from tslearn.clustering import TimeSeriesKMeans, KShape, KernelKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from am4894plots.plots import plot_lines, plot_lines_grid, plot_scatters

In [16]:
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import DBSCAN, AgglomerativeClustering, Birch, MeanShift, AffinityPropagation, OPTICS
from sklearn.metrics import silhouette_samples, silhouette_score

In [46]:
model_cl_dfl = clustering_model('dbscan', 4, spread_rur_nanfree)

0.71


In [48]:
print_graphs(model_cl_dfl, spread_rur_nanfree, 1)

In [45]:
print_graphs(model_cl_dfl, spread_rur_nanfree, 1)

In [36]:
labels_dfl = pd.read_csv('out.csv')

In [38]:
labels_dfl.cluster.value_counts()

2    19
0    15
1    10
3     5
Name: cluster, dtype: int64

In [89]:
labels_dfl = cluster_labels(model_cl_dfl, spread_rur_nanfree)

In [90]:
labels_dfl.to_csv('out.csv', index=False)

In [18]:
def cluster_coint(data, cluster, model, alpha):
    df = data[np.array(cluster_labels(model, data)[cluster_labels(model, data).cluster == cluster].metric)]
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt) 
    

In [19]:
def cluster_adf(data, cluster, model):
    df = data[np.array(cluster_labels(model, data)[cluster_labels(model, data).cluster == cluster].metric)]
    for name, column in df.iteritems():
        adfuller_test(column, name=column.name)
        print('\n')

In [78]:
cluster_adf(spread_rur_nanfree, 1, model_cl_dfl)

    Augmented Dickey-Fuller Test on "spread_0" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.0676
 No. Lags Chosen       = 4
 Critical value 1%     = -3.593
 Critical value 5%     = -2.932
 Critical value 10%    = -2.604
 => P-Value = 0.2578. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "spread_18" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.2977
 No. Lags Chosen       = 0
 Critical value 1%     = -3.578
 Critical value 5%     = -2.925
 Critical value 10%    = -2.601
 => P-Value = 0.015. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "spread_34" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-S

## Стат тесты

In [20]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

In [21]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [22]:
def adjust(val, length= 6): return str(val).ljust(length)
from statsmodels.stats.stattools import durbin_watson

## Var models

In [23]:
def var_model_cluster(data, cluster, model):
    df = data[np.array(cluster_labels(model, data)[cluster_labels(model, data).cluster == cluster].metric)]
    var_model = VAR(df)
    x = var_model.select_order(maxlags=7)
    print(x.summary())
    return x

In [24]:
def dataf(data, cluster, model):
    df = data[np.array(cluster_labels(model, data)[cluster_labels(model, data).cluster == cluster].metric)]
    df.head()
    return df

In [49]:
var_model_cluster(spread_rur_nanfree, 3, model_cl_dfl)

 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      0.000*      0.000*      1.000*      0.000*
1       0.000       0.000       1.000       0.000
2       0.000       0.000       1.000       0.000
3       0.000       0.000       1.000       0.000
4       0.000       0.000       1.000       0.000
5       0.000       0.000       1.000       0.000
6       0.000       0.000       1.000       0.000
7       0.000       0.000       1.000       0.000
-------------------------------------------------


In [25]:
def var_cluster(data, cluster, model, lags):
    df = data[np.array(cluster_labels(model, data)[cluster_labels(model, data).cluster == cluster].metric)]
    var_model = VAR(df)
    x = var_model.fit(lags)
    print(x.summary())
    print(x.test_whiteness())
    return x

In [153]:
var_cluster(spread_rur_nanfree, 3, model_cl_dfl, 2)

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 18, Jun, 2021
Time:                     20:34:59
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -28.4256
Nobs:                     46.0000    HQIC:                  -29.7930
Log likelihood:           432.720    FPE:                5.31960e-14
AIC:                     -30.6120    Det(Omega_mle):     1.82093e-14
--------------------------------------------------------------------
Results for equation spread_7
                  coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------
const                0.004551         0.005622            0.809           0.418
L1.spread_7          0.687907         0.300529            2.289           0.022
L1.spread_9         -0.264291         0.062335           -4.2

##кластеризация

In [26]:
def clustering_model(model, n_clusters, df):
    X = df.transpose().values
    df = pd.DataFrame(X.reshape(df.shape), columns=df.columns, index=df.index)
    if model == 'kshape':
        model = KShape(n_clusters=n_clusters, max_iter=1000, n_init=2).fit(X)
    elif model == 'kmeans':
        model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", max_iter=10, n_init=2).fit(X)
    elif model == 'dtw':
        model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=5, n_init=2).fit(X)
    elif model == 'softdtw':
        model = TimeSeriesKMeans(n_clusters=n_clusters, metric="softdtw", max_iter=5, n_init=2).fit(X )
    elif model == 'kernelkmeans':
        model = KernelKMeans(n_clusters=n_clusters, kernel="gak", max_iter=5, n_init=2).fit(X)
    elif model == 'dbscan':
        model = DBSCAN().fit(X)
    elif model == 'agglomerative':
        model = AgglomerativeClustering(distance_threshold=0.9, n_clusters=None).fit(X)
    elif model == 'birch':
        model = Birch(n_clusters=n_clusters).fit(X)
    elif model == 'meanshift':
        model = MeanShift().fit(X)
    elif model == 'affinity':
        model = AffinityPropagation().fit(X)
    elif model == 'optics':
        model = OPTICS().fit(X)
    else:
        model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", max_iter=10, n_init=2).fit(X)
    silhouette_avg = silhouette_score(X, model.labels_)
    print(round(silhouette_avg,2))
    return model

In [47]:
def print_graphs(model, df, min_n):
    # build helper df to map metrics to their cluster labels
    df_cluster = pd.DataFrame(list(zip(df.columns, model.labels_)), columns=['metric', 'cluster'])

    # make some helper dictionaries and lists
    cluster_metrics_dict = df_cluster.groupby(['cluster'])['metric'].apply(lambda x: [x for x in x]).to_dict()
    cluster_len_dict = df_cluster['cluster'].value_counts().to_dict()
    clusters_all = [cluster for cluster in cluster_len_dict]
    clusters_dropped = [cluster for cluster in cluster_len_dict if cluster_len_dict[cluster]<min_n]
    clusters_final = [cluster for cluster in cluster_len_dict if cluster_len_dict[cluster]>=min_n]
    for cluster_number in clusters_final:
        # get a rough quality score based on the correlation between metrics in the cluster
        x_corr = df[cluster_metrics_dict[cluster_number]].corr().abs().values
        x_corr_mean = round(x_corr[np.triu_indices(x_corr.shape[0],1)].mean(),2)
        # plot each cluster
        plot_title = f'cluster {cluster_number} (quality={x_corr_mean}, n={cluster_len_dict[cluster_number]})'
        plot_lines(df, cols=cluster_metrics_dict[cluster_number], theme=None, title=plot_title)

In [28]:
def cluster_labels(model, df):
    df_cluster = pd.DataFrame(list(zip(df.columns, model.labels_)), columns=['metric', 'cluster'])
    return df_cluster

In [258]:
model_cl_ksb = clustering_model('affinity', 5, spread_rur_ksb)

0.1


C:\Users\03\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:154: FutureWarning:

'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.



In [532]:
print_graphs(model_cl_ksb, spread_rur_ksb, 2)

## Прогнозы
